## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-20-21-51-15 +0000,nyt,U.S. Coast Guard Boards Tanker Carrying Venezu...,https://www.nytimes.com/2025/12/20/us/politics...
1,2025-12-20-21-50-40 +0000,nyt,Epstein and Maxwell Grand Jury Records Are Rel...,https://www.nytimes.com/2025/12/20/us/epstein-...
2,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...
3,2025-12-20-21-45-04 +0000,nypost,Mamdani backs fellow DSA operative in race to ...,https://nypost.com/2025/12/20/us-news/mamdani-...
4,2025-12-20-21-39-50 +0000,wapo,U.S. apprehends another oil tanker amid Trump’...,https://www.washingtonpost.com/national-securi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2423/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,epstein,27
16,trump,25
81,files,18
122,new,13
51,was,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-12-20-04-35-07 +0000,nyt,Epstein Files Timeline: How the Trump Administ...,https://www.nytimes.com/2025/12/19/us/politics...,88
180,2025-12-19-23-51-09 +0000,nyt,Trump Is Rarely Mentioned in New Epstein Files,https://www.nytimes.com/2025/12/19/us/trump-ep...,85
17,2025-12-20-20-09-15 +0000,nypost,Sick new details about Jeffrey Epstein’s crime...,https://nypost.com/2025/12/20/us-news/sick-new...,80
137,2025-12-20-04-24-00 +0000,wsj,Days after President Trump moved to block stat...,https://www.wsj.com/articles/new-york-signs-ai...,75
165,2025-12-20-01-20-00 +0000,wsj,The Justice Department has released files rela...,https://www.wsj.com/politics/policy/justice-de...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,88,2025-12-20-04-35-07 +0000,nyt,Epstein Files Timeline: How the Trump Administ...,https://www.nytimes.com/2025/12/19/us/politics...
182,57,2025-12-19-23-43-00 +0000,wsj,"Rep. Elise Stefanik (R., N.Y.) said she would ...",https://www.wsj.com/politics/elections/elise-s...
155,51,2025-12-20-02-11-25 +0000,nyt,Suspect in Brown and M.I.T. Killings Died 2 Da...,https://www.nytimes.com/2025/12/19/us/suspect-...
41,36,2025-12-20-18-21-29 +0000,nypost,US kills ‘at least 5’ ISIS thugs in Syria in r...,https://nypost.com/2025/12/20/us-news/us-shock...
36,35,2025-12-20-18-38-00 +0000,nypost,Elderly NYC woman found beaten to death near ‘...,https://nypost.com/2025/12/20/us-news/senior-c...
197,33,2025-12-19-22-56-48 +0000,nypost,Rubio warns Ukraine war won’t end in surrender...,https://nypost.com/2025/12/19/world-news/rubio...
60,30,2025-12-20-16-36-01 +0000,nypost,Former Hamas hostage warned leaders of rising ...,https://nypost.com/2025/12/20/world-news/forme...
50,28,2025-12-20-17-32-51 +0000,nypost,Will NYC have a white Christmas this year? For...,https://nypost.com/2025/12/20/us-news/will-nyc...
71,27,2025-12-20-15-32-25 +0000,nypost,Israel’s Benjamin Netanyahu expected to pitch ...,https://nypost.com/2025/12/20/world-news/israe...
133,27,2025-12-20-05-47-00 +0000,wsj,President Trump traveled Friday to North Carol...,https://www.wsj.com/politics/policy/trump-tout...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
